In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [15]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint
import numpy as np
import random
import sys
import io

path = get_file(
    'amlo.txt',
    origin='https://raw.githubusercontent.com/atellez08/ia2/master/LSTM/test.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


581632/580271 [==============================] - 0s 0us/step
corpus length: 569206
total chars: 64


In [16]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 189722
Vectorization...


In [17]:
from keras.models import model_from_json
json_file = open('drive/My Drive/colab/LSTM/amlo8_5.json', 'r')
loaded_model1_json = json_file.read()
json_file.close()
loaded_model1 = model_from_json(loaded_model1_json)
# load weights into new model
loaded_model1.load_weights("drive/My Drive/colab/LSTM/amlo8_5.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [0]:
def predict_loaded(sentence):
    # Function invoked at end of each epoch. Prints generated text.
    print()

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.5]:

        generated = ''
        # sentence = text[start_index: start_index + maxlen]
        # sentence = "vamos a terminar con la corrupción y se "
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = loaded_model1.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [42]:
sentence = input("Introduce un texto de 40 caracteres: ").lower()
if len(sentence) > 40 or len(sentence) < 40:
    print("Tiene que ser 40 caracteres")
else:
    
    predict_loaded(sentence)

Introduce un texto de 40 caracteres: vamos a terminar con la corrupción y se 

----- Generating with seed: "vamos a terminar con la corrupción y se "
vamos a terminar con la corrupción y se está apagado de la constitución. estan también en la estos para el despendinarios, de méxico, se todas estudiando.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 estamos abaja, en el país. ya está admeros a la para todos, el país en el y sablen la tierra.
por eso es maretante por este producción esta es para el confrente un producción la constitución de estas estables de la gasolinas, lo que es margen en la constitución que tentano este año es
